# 15-12-28

Right now the masks for centroid finding are the shape of the contour. PS3 uses a circle centered on the peak of the contour. I'd like to compare these two strategies to see if they give radically different results

In [ ]:
# Import Modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fftpack import fft2,ifft2,fftshift
from speckle_fns import fits_import, circ_filter1
from speckle_fns import deconv0,deconv1, postprocess
from speckle_fns import filter_lpf, filter_hpf, filter_interference
from speckle_fns import fits_view
import sys
import cv2
import math

%matplotlib inline

In [ ]:
# Folder containing FITS files
fileDirectory = '/home/niels/Documents/FITS/'

# Filenames (without extensions) of the Double/Reference pairs
# Preprocessed files must have _PSD terminations on them in the same folder
filenames=[ ['KP330','KP331'], 
            ['KP332','KP333'], 
            ['KP334','KP335'], 
            ['KP336','KP338'], 
            ['KP339','KP340'],
            ['KP341','KP342'] ]

In [ ]:
# Select which set of data to use:
k = 0

# View sample fits images:
imgDoubleStar = fits_view(fileDirectory+filenames[k][0]+'.fits',0)
imgSingleStar = fits_view(fileDirectory+filenames[k][1]+'.fits',0)

plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
plt.imshow(imgDoubleStar)
plt.title('Double Star Image')
plt.subplot(1,2,2)
plt.imshow(imgSingleStar)
plt.title('Reference Star Image')

# Generating Autocorrelogram
# Import FITS file data
psdDoubleStar = fits_import(fileDirectory+filenames[k][0]+'_PSD.fits').astype(float)
psdSingleStar = fits_import(fileDirectory+filenames[k][1]+'_PSD.fits').astype(float)

plt.figure(figsize=(15,15))
plt.imshow(fftshift(np.log10(psdDoubleStar)))
plt.title("Double Star PSD")


In [ ]:
# Perform Deconvolution
constant = 1E-15
imgF = fftshift(deconv1(psdDoubleStar, psdSingleStar, constant))

# Filter PSD:
imgF_filtered = filter_interference(imgF)
imgF_filtered = filter_lpf(imgF_filtered, 10)

# Postprocess
img_autocorr = postprocess(imgF_filtered)

# Plot Image
plt.figure(figsize = (8,8))
plt.imshow(img_autocorr)
plt.title("Autocorr of Filtered Image")
plt.colorbar()


In [ ]:
# Want to find the lowest threshold that we only see the 3 main lobes

# New variable for operating on
autocorr = img_autocorr

# Start thresh at max value of autocorr (assuming that is from central peak)
threshold = np.floor(autocorr.max())

# Calculate values to increment/decrement threshold by depending on height of autocorrelogram
increment_coarse = autocorr.max()/10
increment_fine   = autocorr.max()/100

# Assume we start with 1 contour visible 
num_contours = 1

# Step down threshold until we see the 3 contours we expect
while (num_contours != 3):
    # Decrement the threshold
    threshold = threshold - increment_fine
    
    # Calculate indices in image above threshold
    above_thresh_i = autocorr > threshold
    
    # Create thresholded image
    autocorr_thresh = np.zeros(np.shape(autocorr))
    autocorr_thresh[above_thresh_i] = 255
    autocorr_thresh = autocorr_thresh.astype(np.uint8)
    
    # Find contours of threshold image
    im,contours,hierarchy = cv2.findContours(autocorr_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)
    
    if threshold < 0:
        print("Error in finding 3 contours")
        sys.exit()
        
# Calculate indices in image above threshold
above_thresh_i = autocorr > threshold
    
# Create thresholded image
autocorr_thresh = np.zeros(np.shape(autocorr))
autocorr_thresh[above_thresh_i] = 255
autocorr_thresh = autocorr_thresh.astype(np.uint8)
        
#print(threshold)
#plt.imshow(autocorr_thresh)

# Now we've found our 3 main contours. Keep stepping down until we don't see 3 anymore
# Can be less than 3 if they start to blend together, or more than 3 if other noise in
#  image appears
while (num_contours == 3):
    # Decrement the threshold
    threshold = threshold - increment_fine
    
    # Calculate indices in image above threshold
    above_thresh_i = autocorr > threshold
    
    # Create thresholded image
    autocorr_thresh = np.zeros(np.shape(autocorr))
    autocorr_thresh[above_thresh_i] = 255
    autocorr_thresh = autocorr_thresh.astype(np.uint8)
    
    # Find contours of threshold image
    im,contours,hierarchy = cv2.findContours(autocorr_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)
    
    if threshold < 0:
        print("Error in moving threshold below 3 contours")
        sys.exit()
        
# Calculate indices in image above threshold
above_thresh_i = autocorr > threshold
    
# Create thresholded image
autocorr_thresh = np.zeros(np.shape(autocorr))
autocorr_thresh[above_thresh_i] = 255
autocorr_thresh = autocorr_thresh.astype(np.uint8)
        
#print(threshold)
#plt.imshow(autocorr_thresh)

# Step back up in threshold until we are back to 3 contours
while (num_contours != 3):
    # Decrement the threshold
    threshold = threshold + increment_fine
    
    # Calculate indices in image above threshold
    above_thresh_i = autocorr > threshold
    
    # Create thresholded image
    autocorr_thresh = np.zeros(np.shape(autocorr))
    autocorr_thresh[above_thresh_i] = 255
    autocorr_thresh = autocorr_thresh.astype(np.uint8)
    
    # Find contours of threshold image
    im,contours,hierarchy = cv2.findContours(autocorr_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)    
    
    if threshold > autocorr.max():
        print("Error in moving threshold back up to 3 contours")
        sys.exit()

# Calculate indices in image above threshold
above_thresh_i = autocorr > threshold
    
# Create thresholded image
autocorr_thresh = np.zeros(np.shape(autocorr))
autocorr_thresh[above_thresh_i] = 1
autocorr_thresh = autocorr_thresh.astype(np.uint8)

# Display results
print("Final Threshold = ",threshold)
plt.figure(figsize=(8,8))
plt.imshow(np.multiply(autocorr_thresh,autocorr))

In [ ]:
## Now that we have the contours, we want to find their centroids for sorting purposes
centroid = np.zeros((3,2))

for i in np.arange(3):
    # Calculating Image Moment/Raw Moment of contour
    M = cv2.moments(contours[i])
    
    # Calculate centroid X and Y components 
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    centroid[i] = (cy, cx)
    
## Find the central contour
# Center pixel indices
center = np.divide(np.shape(autocorr),2)
center_distance = np.zeros(3)

# Calculate distance from each centroid to center of image
for i in np.arange(3):
    center_distance[i] = np.linalg.norm(centroid[i] - center)

# Get index of central contour as contour with min distance to center
i = np.where(center_distance == center_distance.min())
i_center = i[0]

# Calculate central contour mask
mask_center = np.zeros(np.shape(autocorr))
cv2.drawContours(mask_center,contours,i_center,(1,1,1),-1)

## Finding indices of side lobes
# Possible indices of side lobes
i_side_lobes = [0,1,2]
# Remove the center lobe index
i_side_lobes.remove(i_center)

# Make empty mask images
mask_side = np.zeros((2,np.shape(autocorr)[0],np.shape(autocorr)[1]))
lobe_side = np.zeros((2,np.shape(autocorr)[0],np.shape(autocorr)[1]))

# Create masks and calculate side lobes
for i in np.arange(2):
    cv2.drawContours(mask_side[i],contours,i_side_lobes[i],(1,1,1),-1)
    lobe_side[i] = (np.multiply(autocorr,mask_side[i]))#.astype(np.uint8)

# Display masked side lobes
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(lobe_side[0])
plt.title("Side Lobe 1")
plt.subplot(1,2,2)
plt.imshow(lobe_side[1])
plt.title("Side Lobe 2")

In [ ]:
## Want to find peak locations of each side lobe image
# Array to hold peak values
lobe_peak = np.zeros((2,2))

# Find peak values
for i in np.arange(2):
    peak = np.where(lobe_side[i] == lobe_side[i].max())
    lobe_peak[i,1] = peak[0][0] # Assigning Y value of peak
    lobe_peak[i,0] = peak[1][0] # Assigning X value of peak
    
# Create images of sidelobes with peaks marked
lobe_side_peak = np.array(lobe_side)

# Put lines on sidelobe 0
lobe_side_peak[0,:,np.round(lobe_peak[0,0]).astype(int)] = 0
lobe_side_peak[0,np.round(lobe_peak[0,1]).astype(int),:] = 0

# Put lines on sidelobe 1
lobe_side_peak[1,:,np.round(lobe_peak[1,0]).astype(int)] = 0
lobe_side_peak[1,np.round(lobe_peak[1,1]).astype(int),:] = 0   

# View peak locations
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(lobe_side_peak[0])
plt.subplot(1,2,2)
plt.imshow(lobe_side_peak[1])

# Print peak locations
print(lobe_peak)

In [ ]:
## Finding radius of circle centered on peak that fits inside contour

lobe_mask_radius = np.zeros(2) # Radius of circles to be calculated 
lobe_mask_radius.fill(math.inf) # Initial value = Infinite

# Calculate for each side lobe
for i in np.arange(2):
    # For each point on contour
    for perimeter_point in contours[i_side_lobes[i]]: 
        # Calculating distance between peak and present contour point
        distance = np.linalg.norm(perimeter_point - lobe_peak[i])
        # Save the minimum value of previously calculated min radius and present calc
        lobe_mask_radius[i] = min([lobe_mask_radius[i],distance])
     
print(lobe_mask_radius)

# 15-12-29

Now that we are finding the minimum radius, lets draw the circles to create the masks and apply the masks

In [ ]:
# Confirming that these circles are centered on the maximum value

# Creating marked images of side lobes
lobe_side_marked = np.zeros((2,512,512))

# Creating fully filled in mask
peak_mask = np.zeros((512,512,3))
peak_mask.fill(1)

# Marking the mask with circles centered on peaks
cv2.circle(peak_mask, (lobe_peak[0,0].astype(np.uint16),lobe_peak[0,1].astype(np.uint16)), lobe_mask_radius[0].astype(np.uint8), (0,0,0), 2)
cv2.circle(peak_mask, (lobe_peak[1,0].astype(np.uint16),lobe_peak[1,1].astype(np.uint16)), lobe_mask_radius[1].astype(np.uint8), (0,0,0), 2)

# Apply the mask to the autocorrelogram
lobe_side_marked[0] = peak_mask[:,:,0]*lobe_side[0]
lobe_side_marked[1] = peak_mask[:,:,0]*lobe_side[1]

# View peak mask
plt.figure(figsize = (8,8))
plt.imshow(peak_mask[:,:,0])

# View marked side lobes
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(lobe_side_marked[0])
plt.subplot(1,2,2)
plt.imshow(lobe_side_marked[1])

In [ ]:
# Masking off the peak centered circles

# Creating marked images of side lobes
lobe_side_masked = np.zeros((2,512,512))

# Creating fully filled in mask
peak_mask = np.zeros((2,512,512,3))

# Marking the mask with circles centered on peaks
cv2.circle(peak_mask[0], (lobe_peak[0,0].astype(np.uint16),lobe_peak[0,1].astype(np.uint16)), lobe_mask_radius[0].astype(np.uint8), (1,1,1), -1)
cv2.circle(peak_mask[1], (lobe_peak[1,0].astype(np.uint16),lobe_peak[1,1].astype(np.uint16)), lobe_mask_radius[1].astype(np.uint8), (1,1,1), -1)

# Apply the mask to the autocorrelogram
lobe_side_masked[0] = peak_mask[0,:,:,0]*lobe_side[0]
lobe_side_masked[1] = peak_mask[1,:,:,0]*lobe_side[1]

# View peak mask
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(peak_mask[0])
plt.subplot(1,2,2)
plt.imshow(peak_mask[1])

# View marked side lobes
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(lobe_side_masked[0])
plt.subplot(1,2,2)
plt.imshow(lobe_side_masked[1])

In [ ]:
## Calculating intensity weighted centroid of side lobes, with circular masking

M00 = np.zeros((2))
M10 = np.zeros((2))
M01 = np.zeros((2))
centroid = np.zeros((2,2))

# Calculate for each side lobe
for lobe in np.arange(2):
    
    # Calculate average
    M00[lobe] = np.sum(lobe_side_masked[lobe])
    
    # Calculate X component
    # Loop through each column
    for column in np.arange(np.shape(lobe_side_masked[lobe])[1]):
        
        M10[lobe] += np.multiply(column,np.sum(lobe_side_masked[lobe,:,column]))
        
        #print('X = ', column, np.multiply(column,np.sum(lobe_side[lobe,:,column])))
    
    # Calculate Y component
    # Loop through each column
    for row in np.arange(np.shape(lobe_side[lobe])[0]):
        
        M01[lobe] += np.multiply(row,np.sum(lobe_side_masked[lobe,row,:]))
    
    # Calculate X and Y Centroid Components of each lobe
    centroid[lobe,0] = M10[lobe]/M00[lobe] # X Component
    centroid[lobe,1] = M01[lobe]/M00[lobe] # Y Component

# Print results of centroid finding
print(M00)
print(M10)
print(M01)
print(centroid[0,0], centroid[0,1])
print(centroid[1,0], centroid[1,1])

# Print original image with centroid locations overlayed
autocorr_cen = np.array(autocorr)
lobe_cen = np.array(lobe_side_masked)

# Put lines on X coordinates of centroids
autocorr_cen[:,np.round(centroid[0,0]).astype(int)] = 0
autocorr_cen[:,np.round(centroid[1,0]).astype(int)] = 0
# Put lines on Y coordinates of centroids
autocorr_cen[np.round(centroid[0,1]).astype(int),:] = 0
autocorr_cen[np.round(centroid[1,1]).astype(int),:] = 0

# Put lines on sidelobe 0
lobe_cen[0,:,np.round(centroid[0,0]).astype(int)] = 0
lobe_cen[0,np.round(centroid[0,1]).astype(int),:] = 0

# Put lines on sidelobe 1
lobe_cen[1,:,np.round(centroid[1,0]).astype(int)] = 0
lobe_cen[1,np.round(centroid[1,1]).astype(int),:] = 0

# Graph all centroid marked plots
plt.figure(figsize=(20,8))

plt.subplot(1,3,1)
plt.imshow(lobe_cen[0])
plt.title("Sidelobe 0 Centroid Location")

plt.subplot(1,3,2)
plt.imshow(lobe_cen[1])
plt.title("Sidelobe 1 Centroid Location")

plt.subplot(1,3,3)
plt.imshow(autocorr_cen)
plt.title("Both Sidelobe Centroid Locations")

centroid_peak = np.array(centroid)

In [ ]:
## Calculating intensity weighted centroid of side lobes, using entire contour

M00 = np.zeros((2))
M10 = np.zeros((2))
M01 = np.zeros((2))
centroid = np.zeros((2,2))

# Calculate for each side lobe
for lobe in np.arange(2):
    
    # Calculate average
    M00[lobe] = np.sum(lobe_side[lobe])
    
    # Calculate X component
    # Loop through each column
    for column in np.arange(np.shape(lobe_side[lobe])[1]):
        
        M10[lobe] += np.multiply(column,np.sum(lobe_side[lobe,:,column]))
        
        #print('X = ', column, np.multiply(column,np.sum(lobe_side[lobe,:,column])))
    
    # Calculate Y component
    # Loop through each column
    for row in np.arange(np.shape(lobe_side[lobe])[0]):
        
        M01[lobe] += np.multiply(row,np.sum(lobe_side[lobe,row,:]))
    
    # Calculate X and Y Centroid Components of each lobe
    centroid[lobe,0] = M10[lobe]/M00[lobe] # X Component
    centroid[lobe,1] = M01[lobe]/M00[lobe] # Y Component

# Print results of centroid finding
print(M00)
print(M10)
print(M01)
print(centroid[0,0], centroid[0,1])
print(centroid[1,0], centroid[1,1])

# Print original image with centroid locations overlayed
autocorr_cen = np.array(autocorr)
lobe_cen = np.array(lobe_side)

# Put lines on X coordinates of centroids
autocorr_cen[:,np.round(centroid[0,0]).astype(int)] = 0
autocorr_cen[:,np.round(centroid[1,0]).astype(int)] = 0
# Put lines on Y coordinates of centroids
autocorr_cen[np.round(centroid[0,1]).astype(int),:] = 0
autocorr_cen[np.round(centroid[1,1]).astype(int),:] = 0

# Put lines on sidelobe 0
lobe_cen[0,:,np.round(centroid[0,0]).astype(int)] = 0
lobe_cen[0,np.round(centroid[0,1]).astype(int),:] = 0

# Put lines on sidelobe 1
lobe_cen[1,:,np.round(centroid[1,0]).astype(int)] = 0
lobe_cen[1,np.round(centroid[1,1]).astype(int),:] = 0

# Graph all centroid marked plots
plt.figure(figsize=(20,8))

plt.subplot(1,3,1)
plt.imshow(lobe_cen[0])
plt.title("Sidelobe 0 Centroid Location")

plt.subplot(1,3,2)
plt.imshow(lobe_cen[1])
plt.title("Sidelobe 1 Centroid Location")

plt.subplot(1,3,3)
plt.imshow(autocorr_cen)
plt.title("Both Sidelobe Centroid Locations")

centroid_contour = np.array(centroid)

In [ ]:
print("Centroids (Sphere centered on peak)", '\n', centroid_peak)
print("Centroids (Contours)", "\n", centroid_contour)

## Results

k = 0: <br />
Centroids (Sphere centered on peak) <br />
 [[ 403.24772626  405.91946966]<br />
 [ 108.75227374  106.08053034]]<br />
Centroids (Contours) <br />
 [[ 403.37878518  405.93153327]<br />
 [ 108.62121482  106.06846673]]<br />
 
k = 1:<br />

Centroids (Sphere centered on peak) <br />
 [[ 220.00967484  255.04554408]<br />
 [ 291.99032516  256.95445592]]<br />
Centroids (Contours) <br />
 [[ 219.52224404  258.60562117]<br />
 [ 292.47775596  253.39437883]]<br />
 
 k = 2: <br />
 Centroids (Sphere centered on peak) <br />
 [[ 469.02557551  506.57034716]<br />
 [  42.97582522    5.06646098]]<br />
Centroids (Contours) <br />
 [[ 469.27102281  506.21833824]<br />
 [  42.75022255    5.38711535]]<br />
 
 k = 3 <br/>
Centroids (Sphere centered on peak) <br/>
 [[ 225.11283002  274.88879163]<br/>
 [ 286.88716998  237.11120837]]<br/>
Centroids (Contours) <br/>
 [[ 226.27968543  274.05319041]<br/>
 [ 285.72031457  237.94680959]]<br/>
 
 k = 4 <br/>
 Centroids (Sphere centered on peak) <br/>
 [[ 282.11470187  275.93283984]<br/>
 [ 229.88529813  236.06716016]]<br/>
Centroids (Contours) <br/>
 [[ 282.26341204  275.74745327]<br/>
 [ 229.73658796  236.25254673]]<br/>
 
 k = 5 <br/>
 Error moving below 3 contours